In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle



    Only loading the 'en' tokenizer.



In [23]:
path = 'Data/'
train_path = 'Train/'
val_path = 'Test/'

In [24]:
# Number of training words
!find {path}{train_path} -name '*.txt' | xargs cat | wc -w

1133172


In [26]:
# Number of validation words
!find {path}{val_path} -name '*.txt' | xargs cat | wc -w

417375


In [27]:
train_files = !ls {path}{train_path}
music = ! cat {path}{train_path}{train_files[42]}
music

[' ridin round town with all the windows down eight track playin all your favrite sounds the rhythm of the bongos fill the park the street musicians tryin to get a startcause its summer summer time is here yes its summer my time of year yes its summer my time of yearstretched out on a blanket in the sand kids of all ages diggin disneyland rappin on the cb radio in your van well give a big ten four to the truckin mancause its summer summer time is here yes its summer my time of year yes its summer my time of yearyoung boys playin stick ball in the street fire hydrants help to beat the heat old man feeding pigeons in the square nighttime finds young lovers walking therecause its summer summer time is here yes its summerin atlantic city or out in malibu or any where between im telling you when you feel those balmy breezes on your face summer time is the best time any placecause its summer summer time is here yes its summer my time of year cause its summer summer time is here yes it is yes

## Tokenize data

In [14]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [15]:
bs = 64
bptt = 80

In [28]:
files = dict(train=train_path, validation=val_path, test=val_path)
md = LanguageModelData.from_text_files(path,
                                       TEXT,
                                       train=train_path,
                                       validation=val_path,
                                       test=val_path,
                                       bs=bs,
                                       bptt=bptt,
                                       min_freq=5)

In [30]:
# batches; # unique tokens in the vocab; # tokens in the training set; # sentences
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(231, 6910, 1, 1192016)

In [31]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'you', 'i', 'the', 'to', 'and', 'me', 'a', 'nt', 'it', 'my']

In [33]:
TEXT.vocab.stoi['you']

2

## Train

In [34]:
em_sz = 60
nh = 300
nl = 3

In [35]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [39]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.10, dropout=0.10, wdrop=0.2, dropoute=0.03, dropouth=0.10)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [37]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       6.00133  5.86945]                                
[ 1.       5.59261  5.42489]                                
[ 2.       5.39982  5.30984]                                
[ 3.       5.15129  5.07369]                                
[ 4.       4.94446  4.91066]                                
[ 5.       4.82815  4.85452]                                
[ 6.       4.78972  4.83948]                                
[ 7.       4.77114  4.80697]                                
[ 8.       4.63997  4.73626]                                
[ 9.       4.52654  4.69132]                                
[ 10.        4.44034   4.67033]                             
[ 11.        4.36855   4.64828]                             
[ 12.        4.33756   4.63598]                             
[ 13.        4.28277   4.62936]                             
[ 14.        4.30507   4.62736]                             



In [40]:
learner.fit(3e-3, 4, wds=1e-6, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=15), HTML(value='')))

[ 0.       6.01394  5.8769 ]                                
[ 1.       5.64478  5.45252]                                
[ 2.       5.46924  5.36473]                                
[ 3.       5.23283  5.095  ]                                
[ 4.       5.03959  4.96805]                                
[ 5.       4.9443   4.90094]                                
[ 6.      4.9044  4.8867]                                   
[ 7.       4.87283  4.82678]                                
[ 8.       4.75775  4.75595]                                
[ 9.       4.66624  4.70453]                                
[ 10.        4.59726   4.67467]                             
[ 11.        4.54458   4.64862]                             
[ 12.        4.49682   4.63122]                             
[ 13.        4.47893   4.62347]                             
[ 14.        4.46012   4.62269]                             



In [41]:
learner.save('increased_dropout_v1')

In [43]:
learner.load('increased_dropout_v1')

In [44]:
m=learner.model

In [45]:
# Define initial word
word = 'all'
pred = TEXT.vocab.stoi[word]
pred = Variable(torch.LongTensor([pred]).cuda())

# How many preds to choose from
top_n = 3
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()

In [46]:
print(word, end=' ')

for _ in range(100):
    # Get predictions from model
    res = m(pred.unsqueeze(0))[0]
    # Select an random word from the top 5 predictions
    prob, preds = res[-1].topk(top_n)
    random_idx = np.random.choice(top_n)
    pred = preds[random_idx]
    word = TEXT.vocab.itos[to_np(pred)[0]]
    print(word, end=' ')

all of my eyes i ve seen you and the world is <unk> i ve seen a way i m not gon make you feel my love and i ve been gone you ll never see me and i m not so <unk> <unk> i m so in <unk> and i know that you are the only <unk> you know that you are the one that you re not in my mind i know i m a friend you re a <unk> <unk> <unk> <unk> and i know you re the <unk> you know that you ll never know that its 